# <center>FOOTBALL DATA</center> <a class="tocSkip">

# PREPARATION <a class="anchor" id="preparation"></a>

## IMPORTING

In [1]:
# IMPORT MODULES
import pandas as pd
import numpy as np
import xlsxwriter
import calendar
import sys
import string

In [2]:
# IMPORT DATA
filename = 'E0.csv'
# https://www.football-data.co.uk/englandm.php
df_original = pd.read_csv(filename)

## PARAMETERS

In [3]:
# PARAMETERS
alphabet = string.ascii_uppercase
table_cols = 26
c_home_goals = 'FTHG'
c_away_goals = 'FTAG'
c_result = 'FTR'
col_diff = 8
col_points = 9
league = 'PREMIER LEAGUE'

## PREPROCESSING

In [4]:
# CONVERT TO DATETIME
df = df_original.copy()
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
n = df.shape[0]
num_cols = df.shape[1]
print(f'There are {n} rows and {num_cols} columns')
df

There are 225 rows and 106 columns


,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2020-09-12,12:30,Fulham,Arsenal,0,3,A,0,1,...,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87
1,E0,2020-09-12,15:00,Crystal Palace,Southampton,1,0,H,1,0,...,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12
2,E0,2020-09-12,17:30,Liverpool,Leeds,4,3,H,3,2,...,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04
3,E0,2020-09-12,20:00,West Ham,Newcastle,0,2,A,0,0,...,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86
4,E0,2020-09-13,14:00,West Brom,Leicester,0,3,A,0,0,...,1.73,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,E0,2021-02-07,12:00,Tottenham,West Brom,2,0,H,0,0,...,2.19,-1.25,1.91,2.02,1.93,2.00,2.02,2.15,1.88,2.01
221,E0,2021-02-07,14:00,Wolves,Leicester,0,0,D,0,0,...,1.64,0.25,1.93,1.97,1.96,1.96,1.99,2.03,1.92,1.96
222,E0,2021-02-07,16:30,Liverpool,Man City,1,4,A,0,0,...,2.12,0.25,1.92,2.01,1.93,2.00,2.02,2.03,1.92,1.97
223,E0,2021-02-07,19:15,Sheffield United,Chelsea,1,2,A,0,1,...,1.87,1.00,2.01,1.92,1.99,1.93,2.10,1.96,2.02,1.86


In [5]:
# REPLACE 'H', 'D', 'A'
df.FTR = df.FTR.replace('H', 'Home win')
df.FTR = df.FTR.replace('D', 'Draw')
df.FTR = df.FTR.replace('A', 'Away win')

In [6]:
df

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,2020-09-12,12:30,Fulham,Arsenal,0,3,Away win,0,1,...,1.84,0.75,2.01,1.89,2.02,1.91,2.13,1.92,2.02,1.87
1,E0,2020-09-12,15:00,Crystal Palace,Southampton,1,0,Home win,1,0,...,1.70,0.25,1.78,2.13,1.79,2.17,1.85,2.18,1.79,2.12
2,E0,2020-09-12,17:30,Liverpool,Leeds,4,3,Home win,3,2,...,2.62,-1.50,1.85,2.05,1.85,2.08,1.90,2.16,1.84,2.04
3,E0,2020-09-12,20:00,West Ham,Newcastle,0,2,Away win,0,0,...,1.92,-0.50,2.03,1.87,2.04,1.88,2.09,1.91,2.02,1.86
4,E0,2020-09-13,14:00,West Brom,Leicester,0,3,Away win,0,0,...,1.73,0.25,1.92,1.98,1.93,1.99,1.95,2.01,1.91,1.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,E0,2021-02-07,12:00,Tottenham,West Brom,2,0,Home win,0,0,...,2.19,-1.25,1.91,2.02,1.93,2.00,2.02,2.15,1.88,2.01
221,E0,2021-02-07,14:00,Wolves,Leicester,0,0,Draw,0,0,...,1.64,0.25,1.93,1.97,1.96,1.96,1.99,2.03,1.92,1.96
222,E0,2021-02-07,16:30,Liverpool,Man City,1,4,Away win,0,0,...,2.12,0.25,1.92,2.01,1.93,2.00,2.02,2.03,1.92,1.97
223,E0,2021-02-07,19:15,Sheffield United,Chelsea,1,2,Away win,0,1,...,1.87,1.00,2.01,1.92,1.99,1.93,2.10,1.96,2.02,1.86


## VARIABLES, FUNCTIONS

In [7]:
# CACHED VARIABLES
home_team = df['HomeTeam']
away_team = df['AwayTeam']
home_goals = df['FTHG']
away_goals = df['FTAG']

In [8]:
# FUNCTIONS
def HomeAppearances(team):
    home_appearances = (home_team == team).sum()
    return home_appearances

def HomeWins(team):
    home_wins = ( (home_team == team) & (home_goals > away_goals) ).sum()
    return home_wins

def HomeDraws(team):
    home_draws = ( (home_team == team) & (home_goals == away_goals) ).sum()
    return home_draws

def HomeLosses(team):
    home_losses = ( (home_team == team) & (home_goals < away_goals) ).sum()
    return home_losses

def HomeFor(team):
    home_for = df[home_team == team]['FTHG'].sum()
    return home_for

def HomeAgainst(team):
    home_against = df[home_team == team]['FTAG'].sum()
    return home_against

def AwayAppearances(team):
    away_appearances = (away_team == team).sum()
    return away_appearances

def AwayWins(team):
    away_wins = ( (away_team == team) & (home_goals < away_goals) ).sum()
    return away_wins

def AwayDraws(team):
    away_draws = ( (away_team == team) & (home_goals == away_goals) ).sum()
    return away_draws

def AwayLosses(team):
    away_losses = ( (away_team == team) & (home_goals > away_goals) ).sum()
    return away_losses

def AwayFor(team):
    away_for = df[away_team == team]['FTAG'].sum()
    return away_for

def AwayAgainst(team):
    away_against = df[away_team == team]['FTHG'].sum()
    return away_against

def SumHomeColumn(team, col):
    home_sum = df[df['HomeTeam'] == team][col].sum()
    return home_sum
    
def SumAwayColumn(team, col):
    away_sum = df[df['AwayTeam'] == team][col].sum()
    return away_sum

In [9]:
# TEST FUNCTIONS
home_sum = SumHomeColumn('Fulham', 'HS')
home_sum
home_team == 'Fulham'

0       True
1      False
2      False
3      False
4      False
       ...  
220    False
221    False
222    False
223    False
224    False
Name: HomeTeam, Length: 225, dtype: bool

In [10]:
home_team == True

0      False
1      False
2      False
3      False
4      False
       ...  
220    False
221    False
222    False
223    False
224    False
Name: HomeTeam, Length: 225, dtype: bool

## METADATA

In [11]:
# GET LIST OF TEAMS
teams = pd.unique(home_team.append(away_team))
num_teams = teams.size
teams

array(['Fulham', 'Crystal Palace', 'Liverpool', 'West Ham', 'West Brom',
       'Tottenham', 'Brighton', 'Sheffield United', 'Everton', 'Leeds',
       'Man United', 'Arsenal', 'Southampton', 'Newcastle', 'Chelsea',
       'Leicester', 'Aston Villa', 'Wolves', 'Burnley', 'Man City'],
      dtype=object)

In [12]:
# DATES
dates = pd.unique(df_original['Date'])
num_dates = dates.size
print(f'There are {num_dates} dates')

There are 74 dates


In [13]:
# VERIFY DATES ARE CHRONOLOGICAL
df_date_changes = df[df['Date'].astype("int64").diff() != 0]
df_date_changes
print(f'Dates are in chronological order: {num_dates == df_date_changes.shape[0]}')

Dates are in chronological order: True


# AGGREGATED STATISTICS <a class="anchor" id="aggregated"></a>

In [14]:
# COUNTS: GOALS
counts_home_goals = df['FTHG'].value_counts().sort_index()
total_home_goals = df['FTHG'].sum()
counts_away_goals = df['FTAG'].value_counts().sort_index()
total_away_goals = df['FTAG'].sum()
total_goals = (df['FTHG'] + df['FTAG']).sum()
counts_goals = (df['FTHG'] + df['FTAG']).value_counts().sort_index()
total_goals
counts_goals

0    18
1    46
2    52
3    48
4    25
5    16
6     8
7     9
8     1
9     2
dtype: int64

In [15]:
# COUNTS: RESULTS
counts_result = df['FTR'].value_counts()
counts_result

Away win    90
Home win    83
Draw        52
Name: FTR, dtype: int64

In [16]:
# KICK-OFF COUNTS
counts_kickoff = df['Time'].value_counts()
counts_kickoff

20:00    47
17:30    27
15:00    24
18:00    23
12:30    18
16:30    18
19:15    15
20:15    15
14:00    14
12:00    14
14:15     5
19:00     2
19:30     2
17:45     1
Name: Time, dtype: int64

In [17]:
# REFEREE COUNTS
counts_referee = df['Referee'].value_counts()
counts_referee

M Atkinson    18
M Oliver      17
M Dean        15
A Taylor      15
C Kavanagh    14
P Tierney     14
A Marriner    14
K Friend      14
C Pawson      13
S Attwell     13
J Moss        13
D Coote       11
G Scott        9
A Madley       9
P Bankes       9
L Mason        9
S Hooper       7
D England      6
R Jones        5
Name: Referee, dtype: int64

# LEAGUE TABLE <a class="anchor" id="league-table"></a>

## CALCULATE FIGURES

In [18]:
# COUNT NUMBER OF HOME APPEARANCES FOR A TEAM
team = teams[0]
appearances = (home_team == team).sum()
appearances

12

In [19]:
# COUNT NUMBER OF HOME WINS FOR A TEAM
home_wins = ( (home_team == team) & (home_goals > away_goals) ).sum()
home_wins

1

In [20]:
# COUNT NUMBER OF HOME DRAWS FOR A TEAM
home_draws = ( (home_team == team) & (home_goals == away_goals) ).sum()
home_draws

4

In [21]:
# COUNT NUMBER OF HOME WINS FOR A TEAM
home_losses = ( (home_team == team) & (home_goals < away_goals) ).sum()
home_losses

7

In [22]:
# COUNT NUMBER OF HOME GOALS SCORED
home_for = df[home_team == teams[0]]['FTHG'].sum()
home_for

7

In [23]:
# COUNT NUMBER OF HOME GOALS CONCEDED
home_against = df[home_team == teams[0]]['FTAG'].sum()
home_against

17

## CONSTRUCT TABLE

In [24]:
# CONSTRUCT LEAGUE TABLE WITHOUT POSITION
league_table = []
for j in range(0, num_teams):
    home_appearances = HomeAppearances(teams[j])
    home_wins = HomeWins(teams[j])
    home_draws = HomeDraws(teams[j])
    home_losses = HomeLosses(teams[j])
    home_for = HomeFor(teams[j])
    home_against = HomeAgainst(teams[j])
    home_diff = home_for - home_against
    home_points = 3*home_wins + home_draws
    away_appearances = AwayAppearances(teams[j])
    away_wins = AwayWins(teams[j])
    away_draws = AwayDraws(teams[j])
    away_losses = AwayLosses(teams[j])
    away_for = AwayFor(teams[j])
    away_against = AwayAgainst(teams[j])
    away_diff = away_for - away_against
    away_points = 3*away_wins + away_draws
    appearances = home_appearances + away_appearances
    wins = home_wins + away_wins
    draws = home_draws + away_draws
    losses = home_losses + away_losses
    goals_for = home_for + away_for
    goals_against = home_against + away_against
    diff = goals_for - goals_against
    points = 3*wins + draws
    
    row_table = ['', teams[j], appearances, wins, draws, losses, goals_for, goals_against, diff, points, \
                home_appearances, home_wins, home_draws, home_losses, home_for, home_against, home_diff, home_points, \
                away_appearances, away_wins, away_draws, away_losses, away_for, away_against, away_diff, away_points]
    league_table.append(row_table)

In [25]:
# CONVERT TO NUMPY ARRAY
league_table = np.asarray(league_table)

In [26]:
# SORT TABLE BY DESCENDING POINTS
idx_points = league_table[:, col_points].argsort()[::-1]
league_table = league_table[idx_points]
league_table

array([['', 'Man City', '22', '15', '5', '2', '43', '14', '29', '50',
        '11', '8', '2', '1', '22', '7', '15', '26', '11', '7', '3', '1',
        '21', '7', '14', '24'],
       ['', 'Man United', '23', '13', '6', '4', '49', '30', '19', '45',
        '12', '5', '3', '4', '25', '18', '7', '18', '11', '8', '3', '0',
        '24', '12', '12', '27'],
       ['', 'Leicester', '23', '13', '4', '6', '39', '25', '14', '43',
        '11', '5', '1', '5', '16', '15', '1', '16', '12', '8', '3', '1',
        '23', '10', '13', '27'],
       ['', 'Liverpool', '23', '11', '7', '5', '44', '29', '15', '40',
        '12', '7', '2', '3', '22', '14', '8', '23', '11', '4', '5', '2',
        '22', '15', '7', '17'],
       ['', 'Chelsea', '23', '11', '6', '6', '38', '24', '14', '39',
        '11', '5', '4', '2', '21', '11', '10', '19', '12', '6', '2', '4',
        '17', '13', '4', '20'],
       ['', 'West Ham', '23', '11', '6', '6', '34', '28', '6', '39',
        '11', '5', '3', '3', '16', '14', '2', '18'

In [27]:
# ADD POSITION, DEAL WITH TIES
# Points, the goal difference, then goals scored. Then tied
j = 0
while j < teams.size-1: # Selection sort needs to be implemented a min of n-1 times
    for i in range(0, num_teams-1):
        if league_table[i][col_points] == league_table[i+1][col_points]:
            if league_table[i][col_diff] == league_table[i+1][col_diff]:
                if league_table[i][col_for] == league_table[i+1][col_for]:
                    league_table[i][0] = i+1 # both have same position
                    league_table[i+1][0] = i+1
                elif int(league_table[i][col_for]) > int(league_table[i+1][col_for]):
                    league_table[i][0] = i+1
                    league_table[i+1][0] = i+2
            elif int(league_table[i][col_diff]) > int(league_table[i+1][col_diff]):
                league_table[i][0] = i+1
                league_table[i+1][0] = i+2
        if int(league_table[i][col_points]) > int(league_table[i+1][col_points]):
            league_table[i][0] = i+1
            league_table[i+1][0] = i+2
    j += 1

In [28]:
# SORT BY LEAGUE POSITION
idx_position = league_table[:, 0].astype("int64").argsort()
league_table = league_table[idx_position]
league_table

array([['1', 'Man City', '22', '15', '5', '2', '43', '14', '29', '50',
        '11', '8', '2', '1', '22', '7', '15', '26', '11', '7', '3', '1',
        '21', '7', '14', '24'],
       ['2', 'Man United', '23', '13', '6', '4', '49', '30', '19', '45',
        '12', '5', '3', '4', '25', '18', '7', '18', '11', '8', '3', '0',
        '24', '12', '12', '27'],
       ['3', 'Leicester', '23', '13', '4', '6', '39', '25', '14', '43',
        '11', '5', '1', '5', '16', '15', '1', '16', '12', '8', '3', '1',
        '23', '10', '13', '27'],
       ['4', 'Liverpool', '23', '11', '7', '5', '44', '29', '15', '40',
        '12', '7', '2', '3', '22', '14', '8', '23', '11', '4', '5', '2',
        '22', '15', '7', '17'],
       ['5', 'Chelsea', '23', '11', '6', '6', '38', '24', '14', '39',
        '11', '5', '4', '2', '21', '11', '10', '19', '12', '6', '2', '4',
        '17', '13', '4', '20'],
       ['6', 'West Ham', '23', '11', '6', '6', '34', '28', '6', '39',
        '11', '5', '3', '3', '16', '14', '2'

## GENERALISE TO FUNCTIONS

In [29]:
# FUNCTIONS
# CONSTRUCT LEAGUE TABLE WITHOUT POSITION
def CreateFirstTable(teams):
    league_table = []
    for j in range(0, teams.size):
        home_appearances = HomeAppearances(teams[j])
        home_wins = HomeWins(teams[j])
        home_draws = HomeDraws(teams[j])
        home_losses = HomeLosses(teams[j])
        home_for = HomeFor(teams[j])
        home_against = HomeAgainst(teams[j])
        home_diff = home_for - home_against
        home_points = 3*home_wins + home_draws
        away_appearances = AwayAppearances(teams[j])
        away_wins = AwayWins(teams[j])
        away_draws = AwayDraws(teams[j])
        away_losses = AwayLosses(teams[j])
        away_for = AwayFor(teams[j])
        away_against = AwayAgainst(teams[j])
        away_diff = away_for - away_against
        away_points = 3*away_wins + away_draws
        appearances = home_appearances + away_appearances
        wins = home_wins + away_wins
        draws = home_draws + away_draws
        losses = home_losses + away_losses
        goals_for = home_for + away_for
        goals_against = home_against + away_against
        diff = goals_for - goals_against
        points = 3*wins + draws

        row_table = ['', teams[j], appearances, wins, draws, losses, goals_for, goals_against, diff, points, \
                    home_appearances, home_wins, home_draws, home_losses, home_for, home_against, home_diff, home_points, \
                    away_appearances, away_wins, away_draws, away_losses, away_for, away_against, away_diff, away_points]
        league_table.append(row_table)
    league_table = np.asarray(league_table)
    return league_table

# SORT TABLE BY DESCENDING POINTS
def SortTableByPoints(league_table):
    idx_points = league_table[:, col_points].argsort()[::-1]
    league_table = league_table[idx_points]
    return league_table

# ADD POSITION, DEAL WITH TIES
def AddPositionToTable(league_table):
    j = 0
    while j < league_table.shape[0]-1: # Selection sort needs to be implemented a min of n-1 times
        for i in range(0, league_table.shape[0]-1):
            if league_table[i][col_points] == league_table[i+1][col_points]:
                if league_table[i][col_diff] == league_table[i+1][col_diff]:
                    if league_table[i][col_for] == league_table[i+1][col_for]:
                        league_table[i][0] = i+1 # both have same position
                        league_table[i+1][0] = i+1
                    elif int(league_table[i][col_for]) > int(league_table[i+1][col_for]):
                        league_table[i][0] = i+1
                        league_table[i+1][0] = i+2
                elif int(league_table[i][col_diff]) > int(league_table[i+1][col_diff]):
                    league_table[i][0] = i+1
                    league_table[i+1][0] = i+2
            if int(league_table[i][col_points]) > int(league_table[i+1][col_points]):
                league_table[i][0] = i+1
                league_table[i+1][0] = i+2
        j += 1
    return league_table

# SORT BY LEAGUE POSITION
def SortTableByPosition(league_table):
    idx_position = league_table[:, 0].astype("int64").argsort()
    league_table = league_table[idx_position]
    return league_table

In [30]:
# CREATE FINAL TABLE
def CreateTable(teams):
    league_table = CreateFirstTable(teams)
    league_table = SortTableByPoints(league_table)
    league_table = AddPositionToTable(league_table)
    league_table = SortTableByPosition(league_table)
    return league_table

In [31]:
# TEST CREATETABLE
table_PL = CreateTable(teams)
table_PL

array([['1', 'Man City', '22', '15', '5', '2', '43', '14', '29', '50',
        '11', '8', '2', '1', '22', '7', '15', '26', '11', '7', '3', '1',
        '21', '7', '14', '24'],
       ['2', 'Man United', '23', '13', '6', '4', '49', '30', '19', '45',
        '12', '5', '3', '4', '25', '18', '7', '18', '11', '8', '3', '0',
        '24', '12', '12', '27'],
       ['3', 'Leicester', '23', '13', '4', '6', '39', '25', '14', '43',
        '11', '5', '1', '5', '16', '15', '1', '16', '12', '8', '3', '1',
        '23', '10', '13', '27'],
       ['4', 'Liverpool', '23', '11', '7', '5', '44', '29', '15', '40',
        '12', '7', '2', '3', '22', '14', '8', '23', '11', '4', '5', '2',
        '22', '15', '7', '17'],
       ['5', 'Chelsea', '23', '11', '6', '6', '38', '24', '14', '39',
        '11', '5', '4', '2', '21', '11', '10', '19', '12', '6', '2', '4',
        '17', '13', '4', '20'],
       ['6', 'West Ham', '23', '11', '6', '6', '34', '28', '6', '39',
        '11', '5', '3', '3', '16', '14', '2'

# TABLE OF GOALS SCORED / CONCEDED <a class="anchor" id="table-goals"></a>

In [32]:
# LOCAL PARAMETERS
col_avg_for = 2
col_avg_diff = 4

In [33]:
# FUNCTIONS
def TableGoals(teams):
    table_goals = []
    for j in range(0, num_teams):
        home_appearances = HomeAppearances(teams[j])
        home_for = HomeFor(teams[j])
        home_against = HomeAgainst(teams[j])
        home_diff = home_for - home_against
        home_for_avg = home_for / home_appearances
        home_against_avg = home_against / home_appearances
        home_diff_avg = (home_for - home_against) / home_appearances
        away_appearances = AwayAppearances(teams[j])
        away_for = AwayFor(teams[j])
        away_against = AwayAgainst(teams[j])
        away_diff = away_for - away_against
        away_for_avg = away_for / away_appearances
        away_against_avg = away_against / away_appearances
        away_diff_avg = (away_for - away_against) / away_appearances
        all_appearances = home_appearances + away_appearances
        all_for = home_for + away_for
        all_against = home_against + away_against
        all_diff = all_for - all_against
        all_for_avg = all_for / all_appearances
        all_against_avg = all_against / all_appearances
        all_diff_avg = (all_for - all_against) / all_appearances
        
        row_table = ['', teams[j], all_for_avg, all_against_avg, all_diff_avg, all_appearances, all_for, all_against, all_diff, \
                 home_for_avg, home_against_avg, home_diff_avg, home_appearances, home_for, home_against, home_diff, \
                 away_for_avg, away_against_avg, away_diff_avg, away_appearances, away_for, away_against, away_diff]
        table_goals.append(row_table)
    table_goals = np.asarray(table_goals)
    return table_goals

def SortTableByAvgDiff(table_goals):
    idx_diff = table_goals[:, col_avg_diff].argsort()[::-1]
    table_goals = table_goals[idx_diff]
    return table_goals

def AddPositionToGoalsTable(table_goals):
    k = 0
    while k < league_table.shape[0]-1: # Selection sort needs to be implemented a min of n-1 times
        for j in range(0, num_teams-1):
            if table_goals[j][col_avg_diff] == table_goals[j+1][col_avg_diff]:
                if table_goals[j][col_avg_for] == table_goals[j+1][col_avg_for]:
                    print("ASDAS")
                    table_goals[j][0] = j+1 # Both have same position
                    table_goals[j+1][0] = j+1
                elif table_goals[j][col_avg_for] > table_goals[j+1][col_avg_for]:
                    table_goals[j][0] = j+1
                    table_goals[j+1][0] = j+2
            elif table_goals[j][col_avg_diff] > table_goals[j+1][col_avg_diff]:
                table_goals[j][0] = j+1
                table_goals[j+1][0] = j+2
        k += 1
    return table_goals

In [34]:
# CREATE FINAL GOALS TABLE
def CreateGoalsTable(teams):
    table_goals = TableGoals(teams)
    table_goals = SortTableByAvgDiff(table_goals)
    table_goals = AddPositionToGoalsTable(table_goals)
    table_goals = SortTableByPosition(table_goals)
    return table_goals

In [35]:
# TEST CREATEGOALTABLE FUNCTION
table_goals = CreateGoalsTable(teams)
table_goals

array([['1', 'Man City', '1.9545454545454546', '0.6363636363636364',
        '1.3181818181818181', '22', '43', '14', '29', '2.0',
        '0.6363636363636364', '1.3636363636363635', '11', '22', '7',
        '15', '1.9090909090909092', '0.6363636363636364',
        '1.2727272727272727', '11', '21', '7', '14'],
       ['2', 'Man United', '2.130434782608696', '1.3043478260869565',
        '0.8260869565217391', '23', '49', '30', '19',
        '2.0833333333333335', '1.5', '0.5833333333333334', '12', '25',
        '18', '7', '2.1818181818181817', '1.0909090909090908',
        '1.0909090909090908', '11', '24', '12', '12'],
       ['3', 'Liverpool', '1.9130434782608696', '1.2608695652173914',
        '0.6521739130434783', '23', '44', '29', '15',
        '1.8333333333333333', '1.1666666666666667', '0.6666666666666666',
        '12', '22', '14', '8', '2.0', '1.3636363636363635',
        '0.6363636363636364', '11', '22', '15', '7'],
       ['4', 'Tottenham', '1.6363636363636365', '1.0',
        '

# EXPORTING <a class="anchor" id="exporting"></a>

## PARAMETERS, FORMATTING, FUNCTIONS

In [36]:
# PARAMETERS
headings_league_table = ['Position', 'Team', 'Played', 'Wins', 'Draws', 'Losses', 'For', 'Against', 'Diff', 'Points', \
                         'Played', 'Wins', 'Draws', 'Losses', 'For', 'Against', 'Diff', 'Points', \
                         'Played', 'Wins', 'Draws', 'Losses', 'For', 'Against', 'Diff', 'Points']
headings_table_goals = ['Position', 'Team', 'For avg', 'Against avg', 'Diff avg', 'Played', 'For', 'Against', 'Diff', \
                       'For avg', 'Against avg', 'Diff avg', 'Played', 'For', 'Against', 'Diff', \
                       'For avg', 'Against avg', 'Diff avg', 'Played', 'For', 'Against', 'Diff']

In [37]:
# CELL FORMATTING
def VariablesCellFormatting(workbook):
    global cf_none
    cf_none = workbook.add_format()
    global cf_bold
    cf_bold = workbook.add_format({'bold': True})
    global cf_italic
    cf_italic = workbook.add_format({'italic': True})
    global cf_0dp
    cf_0dp = workbook.add_format({'num_format': '0'})
    global cf_2dp
    cf_2dp = workbook.add_format({'num_format': '0.00'})
    global cf_left
    cf_left = workbook.add_format({'align': 'left'})
    global cf_right
    cf_right = workbook.add_format({'align': 'right'})
    global cf_percent
    cf_percent = workbook.add_format({'num_format': 9})
    global cf_italic_percent
    cf_italic_percent = workbook.add_format({'italic': True, 'num_format': 9})

def CellFormatting():
    global cfs_league_table
    cfs_league_table = [cf_none, cf_none, cf_none, cf_none, cf_none, cf_none, cf_none, cf_none, cf_none, cf_none, \
                  cf_none, cf_none, cf_none, cf_none, cf_none, cf_none, cf_none, cf_none, \
                  cf_none, cf_none, cf_none, cf_none, cf_none, cf_none, cf_none, cf_none]
    global cfs_table_goals
    cfs_table_goals = [cf_none, cf_none, cf_2dp, cf_2dp, cf_2dp, cf_none, cf_none, cf_none, cf_none, \
                  cf_2dp, cf_2dp, cf_2dp, cf_none, cf_none, cf_none, cf_none, \
                  cf_2dp, cf_2dp, cf_2dp, cf_none, cf_none, cf_none, cf_none]

def ColumnWidth(worksheet, default, team):
    column_widths = []
    for i in range(0, table_cols):
        if i == 1: # Teams column
            column_widths.append(team)
        else:
            column_widths.append(default)
        worksheet.set_column(i, i, column_widths[i])

In [38]:
# FUNCTIONS
def CreateWorkbook(number=''):
    filename = f'Football data{number}.xlsx'
    workbook = xlsxwriter.Workbook(filename, {'nan_inf_to_errors': True})
    return workbook

def CreateWorksheets(workbook, sheetnames):
    worksheets = []
    for i in range(0, len(sheetnames)):
        worksheet = workbook.add_worksheet(sheetnames[i])
        worksheets.append(worksheet)
    return worksheets

def WriteMetadata(worksheet, row, col):
    quantities = ['Matches', 'Home goals', 'Away goals', 'Total goals', \
                       'Avg home goals', 'Avg away goals', 'Avg goals', 'Home advantage']
    values = [n, total_home_goals, total_away_goals, total_goals, \
                   total_home_goals / n, total_away_goals / n, total_goals / n, total_home_goals/total_away_goals]
    cfs = [cf_none, cf_none, cf_none, cf_none, cf_2dp, cf_2dp, cf_2dp, cf_2dp]
    headings = ['Quantity', 'Value']
    worksheet.write(row, col, f'{league} ANALYSIS', cf_bold)
    worksheet.write(row+2, col, 'OVERALL FIGURES', cf_bold)
    for i in range(len(headings)):
        worksheet.write(row+3, col+i, headings[i], cf_bold)
    for i in range(len(quantities)):
        worksheet.write(row+4+i, col, quantities[i])
        worksheet.write(row+4+i, col+1, values[i], cfs[i])

def WriteCounts(table, worksheet, heading: str, row, col):
    headings = [heading]
    headings.append('Count')
    headings.append('Percent')
    row_0 = row
    worksheet.write(row, col, f'COUNT: {heading.upper()}', cf_bold)
    for i in range(len(headings)):
        worksheet.write(row+1, col+i, headings[i], cf_bold)
    for i in range(table.size):
        worksheet.write(row+2, col, table.index[i], cf_left)
        worksheet.write(row+2, col+1, table[i])
        worksheet.write(row+2, col+2, '={0}{1}/{0}{2}'.format(alphabet[col+1], row+3, row_0+3+table.size), cf_percent)
        row += 1
    formula_sum = '=SUM(INDIRECT("{0}"&{1}&":"&"{0}"&{2}))'
    total_formulas = ['Total:', formula_sum, formula_sum]
    cfs = [cf_italic, cf_italic, cf_italic_percent]
    for i in range(0, len(headings)):
        worksheet.write(row+2, col+i, total_formulas[i]\
                        .format(alphabet[i], row+3-table.size, row+2), cfs[i])

def WriteTable(worksheet, table, headings, league, row, col, cfs):
    worksheet.write(row, col, f'{league} TABLE', cf_bold)
    worksheet.write(row+1, col+2,'HOME AND AWAY', cf_bold)
    worksheet.write(row+1, col+10,'HOME', cf_bold)
    worksheet.write(row+1, col+18,'AWAY', cf_bold)
    for j in range(len(headings)):
        worksheet.write(row+2, col+j, headings[j], cf_bold)
    for i in range(num_teams):
        for j in range(len(headings)):
            if j == 1: # skip team name
                worksheet.write(row+3+i, col+j, table[i][j], cfs[j])
            else:
                worksheet.write(row+3+i, col+j, float(table[i][j]), cfs[j])

## EXPORTING

In [39]:
# WRITE TO EXCEL
number = ''
sheetnames = ['Overall', 'Tables']

workbook = CreateWorkbook(number)
worksheets = CreateWorksheets(workbook, sheetnames)
VariablesCellFormatting(workbook)
CellFormatting()
ColumnWidth(worksheets[1], 4, 14)

WriteMetadata(worksheets[0], 0, 0)
WriteCounts(counts_result, worksheets[0], 'Result', 15, 0)
WriteCounts(counts_goals, worksheets[0], 'Goals', 25, 0)
WriteCounts(counts_kickoff, worksheets[0], 'Kickoff time', 40, 0)
WriteCounts(counts_referee, worksheets[0], 'Referee', 60, 0)
WriteTable(worksheets[1], table_PL, headings_league_table, f'{league} TABLE', 0, 0, cfs_league_table)
WriteTable(worksheets[1], table_goals, headings_table_goals, f'{league} GOALS TABLE', 30, 0, cfs_table_goals)

workbook.close()